# cancer_rename

> Please add a description. Don't forgot to edit the below cell!!

In [ ]:
#| default_exp cancer_rename

In [ ]:
#| hide

def colab_is_true():

    try: 
        from google.colab import drive

        return True 
    except ModuleNotFoundError:
        return False

def setup_colab():
    import os
    from google.colab import drive
    drive.mount('/content/drive')
    os.system('unzip -q "/content/drive/My Drive/archive (1).zip"')
    os.chdir('drive/MyDrive/cancer-proj')
    os.system('git pull')
    os.system('pip install -qU -e .')
    os.system('pip install -qU nbdev')
    os.system('nbdev_install_quarto')

if __name__ == "__main__":
    on_colab = colab_is_true()
    if on_colab:
        setup_colab()

In [ ]:
#| hide

from nbdev.showdoc import *

## Predictions given xval and yval:

In [ ]:
#| hide
from fastai.vision.all import *
#| export 
from cancer_proj.cancer_dataloading import *
from cancer_proj.cancer_metrics import *

In [ ]:
# class main_train:
#     """Instantiate and (optionally) train the encoder. Then fine-tune the supervised model. 
#     Outputs metrics on validation data"""

#     def __init__(self,
#                  aug_pipelines, #the aug pipeline for self-supervised learning
#                  aug_pipelines_tune, #the aug pipeline for supervised learning
#                  aug_pipelines_test, #test (or valid) time augmentations 
#                  initial_weights=None, #Which initial weights to use
#                  pretrain=None, #Whether to fit BT
#                  fit_policy=None, #policy for fitting BT
#                  numfit=None,
#                  end=False

#                  ):
#                  store_attr()


#                  #self.encoder_path = f'/content/drive/My Drive/models/baselineencoder_initial_weights={self.initial_weights}_pretrain={self.pretrain}.pth'
#                  #self.tuned_model_path = f'/content/drive/My Drive/models/baselinefinetuned_initial_weights={self.initial_weights}_pretrain={self.pretrain}.pth'


#     def train_encoder(self):


#         bt_model,encoder = create_model(which_model=self.initial_weights)


#         if self.pretrain:

#             learn = Learner(dls_train,bt_model, cbs=[BarlowTwins(self.aug_pipelines,n_in=n_in,lmb=1/ps,print_augs=False)])

#             if self.fit_policy == 'fit_one_cycle':

#                 learn.fit_one_cycle(num_epochs)

#             elif self.fit_policy == 'fit':
#                 #learn.fit(num_epochs)
#                 learn.fit(num_epochs)

#             elif self.fit_policy == 'fine_tune':
#                 learn.fine_tune(num_epochs)

#             #torch.save(encoder.state_dict(), self.encoder_path)

#         #self.encoder=encoder
#         self.encoder = bt_model.encoder

#         #return encoder

#     def fine_tune(self):

#         encoder = pickle.loads(pickle.dumps(self.encoder)) #We might want to pretrain once and fine tune several times (varying e.g. tune augs)
  
#         model = LinearModel(encoder=encoder,indim=2048,outdim=9)
        
#         learn = Learner(dls_tune,model,cbs = [LinearBt(aug_pipelines=self.aug_pipelines_tune,n_in=n_in)],wd=0.0)


#         #We can do an ablation where we test bt_pretrain initial weights with fine_tune as well, 
#         #in the case of pretrain=False

#         #if self.initial_weights == 'supervised_pretrain': assert False

#         if self.initial_weights == 'no_pretrain' or self.initial_weights =='supervised_pretrain':
#             learn.fine_tune(self.numfit)

#         elif self.initial_weights == 'bt_pretrain':
#             learn.fit(self.numfit)
    
#         else:
#             raise Exception('Initial weights not of required form')

#         scores,preds, acc = predict_model(xval,yval,model,numavg=3)

#         #metrics dict will have f1 score, auc etc etc
#         metrics = classification_report_wrapper(preds, yval, vocab, print_report=False)
#         metrics['acc'] = acc
#         #metrics['scores'] = scores
#         #metrics['preds'] = preds

#         auc_dict = plot_roc(yval,preds,vocab,print_plot=False)
#         metrics['auc_dict'] = auc_dict
        
#         #torch.save(model.state_dict(), self.tuned_model_path)
#         return metrics #

#     def __call__(self):

  
#         self.train_encoder() #train (or extract) the encoder
#         metrics = self.fine_tune()
        
#         return metrics

# if __name__ == '__main__':

#     torch.cuda.empty_cache()
#     foo='hi'

#     #assert False

#     #Main for training. Essentially a map from hyperparameters to validation acc (which can eventually be a dictionary etc)
#     numfit=20
#     numavg=1
#     num_epochs=20

#     pretrain = True
#     initial_weights = 'supervised_pretrain'
#     fit_policy = 'fit'

#     avg_acc = 0 
#     for _ in range(numavg):
    
#         main = Main(initial_weights=initial_weights,
#                         pretrain=pretrain,
#                         aug_pipelines=aug_pipelines,
#                         aug_pipelines_tune=aug_pipelines_tune,
#                         aug_pipelines_test=aug_pipelines_test,
#                         fit_policy = fit_policy, #i.e. we use fixed policy for these
#                         numfit=numfit
#                     )
    
#         metrics_false_pretrain = main()

#         avg_acc +=  metrics_false_pretrain['acc']

#         print(metrics_false_pretrain['acc'])

#         del main

#     avg_acc *= 1/numavg

#     print(f'avg_acc is {avg_acc:.3f}')

#     print(f'initial_weights = {initial_weights}, pretrain={pretrain}')



In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()